In [102]:
%reload_ext autoreload
%autoreload 2

In [103]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

import torch.optim as optim
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display,clear_output

import warnings
warnings.filterwarnings("ignore")

In [104]:
from dataset import HelocDataset
from experiments import Benchmarking
from utils.logger_config import setup_logger
from tqdm import tqdm
from models.wrapper import PYTORCH_MODELS

logger = setup_logger()

In [105]:
from dataset import dataset_loader

In [106]:
name = 'heloc'
dataset_ares = dataset_loader(name, data_path='data/', dropped_features=[], n_bins=None)

In [107]:
from experiments.counterfactual import *
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from models import PyTorchDNN, PyTorchLinearSVM, PyTorchRBFNet, PyTorchLogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.gaussian_process import GaussianProcessClassifier

In [108]:
dataset = HelocDataset(dataset_ares=dataset_ares)

input_dim = dataset.get_dataframe().shape[1] - 1
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

counterfactual_algorithms = [
    # 'DiCE',
    # 'DisCount',
    'GlobeCE',
    # 'AReS',
    # 'KNN',
]

experiment = Benchmarking(
    dataset=dataset,
    models=[
        # (GaussianProcessClassifier(),'sklearn'),
        # (PyTorchLogisticRegression(input_dim=input_dim), 'PYT'),
        # (PyTorchDNN(input_dim=input_dim), 'PYT'),
        # (PyTorchRBFNet(input_dim=input_dim, hidden_dim=input_dim), 'PYT'),
        # (PyTorchLinearSVM(input_dim=input_dim), 'PYT'),
        # (RandomForestClassifier(n_estimators=10), 'sklearn'), 
        # (GradientBoostingClassifier(n_estimators=10), 'sklearn'), 
        # (AdaBoostClassifier(n_estimators=10), 'sklearn'), 
    ],
    shapley_methods=[
        "Train_Distri",
        "CF_UniformMatch",
        # "CF_ExactMatch",
        "CF_SingleMatch",
        # "CF_OTMatch",
        "CF_OTMatch_2.0", 
        # "CF_OTMatch_50.0",
        # "CF_OTMatch_100.0",
        # "CF_OTMatch_10.0",
    ],
    distance_metrics=[
        'optimal_transport',
        'mean_difference',
        'median_difference',
        # 'max_mean_discrepancy',
    ]
)

experiment.train_and_evaluate_models(random_state=seed)
experiment.models_performance()

logger.info("\n\n------Compute Counterfactuals------")
sample_num = 1000
model_counterfactuals = {}
for model, model_name in zip(experiment.models, experiment.model_names):
    model_counterfactuals[model_name] = {}

    for algorithm in counterfactual_algorithms:
        if algorithm == 'DisCount' and model_name not in PYTORCH_MODELS:
            logger.info(f'Skipping {algorithm} for {model_name} due to incompatability')
            continue
        logger.info(f'Computing {model_name} counterfactuals with {algorithm}')
        function_name = f"compute_{algorithm}_counterfactuals"
        try:
            func = globals()[function_name]
            model_counterfactuals[model_name][algorithm] = func(
                experiment.X_test,
                model = model,
                target_name = experiment.dataset.target_name,
                sample_num = sample_num,
                experiment=experiment,
            )
        except KeyError:
            print(f"Function {function_name} is not defined.")

INFO:root:PyTorchLinearSVM accuracy: 0.740253164556962
INFO:root:

------Compute Counterfactuals------
INFO:root:Computing PyTorchLinearSVM counterfactuals with GlobeCE


['ExternalRiskEstimate' 'MSinceOldestTradeOpen'
 'MSinceMostRecentTradeOpen' 'AverageMInFile' 'NumSatisfactoryTrades'
 'NumTrades60Ever2DerogPubRec' 'NumTrades90Ever2DerogPubRec'
 'PercentTradesNeverDelq' 'MSinceMostRecentDelq'
 'MaxDelq2PublicRecLast12M' 'MaxDelqEver' 'NumTotalTrades'
 'NumTradesOpeninLast12M' 'PercentInstallTrades'
 'MSinceMostRecentInqexcl7days' 'NumInqLast6M' 'NumInqLast6Mexcl7days'
 'NetFractionRevolvingBurden' 'NetFractionInstallBurden'
 'NumRevolvingTradesWBalance' 'NumInstallTradesWBalance'
 'NumBank2NatlTradesWHighUtilization' 'PercentTradesWBalance']


100%|██████████| 1000/1000 [00:00<00:00, 1694.10it/s]


In [110]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

logger.info("\n\n------Compute Shapley Values------")
experiment.compute_intervention_policies(
    model_counterfactuals=model_counterfactuals,
);

INFO:root:

------Compute Shapley Values------
INFO:root:Shapley values for PyTorchLinearSVM using Train_Distri with counterfactual by GlobeCE


  0%|          | 0/105 [00:00<?, ?it/s]

INFO:root:Shapley values for PyTorchLinearSVM using CF_UniformMatch with counterfactual by GlobeCE


  0%|          | 0/105 [00:00<?, ?it/s]

INFO:root:Shapley values for PyTorchLinearSVM using CF_SingleMatch with counterfactual by GlobeCE
INFO:root:Shapley values for PyTorchLinearSVM using CF_OTMatch_2.0 with counterfactual by GlobeCE


In [111]:
experiment.md_baseline = False

In [134]:
experiment.distance_metrics = [
    'max_mean_discrepancy'
]

In [145]:
logger.info("\n\n------Evaluating Distance Performance Under Interventions------")
experiment.evaluate_distance_performance_under_interventions(
    intervention_num_list=range(0,801,20),
    trials_num=100,
    replace=False,
)

INFO:root:

------Evaluating Distance Performance Under Interventions------
INFO:root:Policy for PyTorchLinearSVM using Train_Distri with counterfactual by GlobeCE
INFO:root:Computing max_mean_discrepancy for (PyTorchLinearSVM, GlobeCE, Train_Distri)
100%|██████████| 100/100 [00:05<00:00, 17.26it/s]
INFO:root:Policy for PyTorchLinearSVM using CF_UniformMatch with counterfactual by GlobeCE
INFO:root:Computing max_mean_discrepancy for (PyTorchLinearSVM, GlobeCE, CF_UniformMatch)
100%|██████████| 100/100 [00:06<00:00, 16.35it/s]
INFO:root:Policy for PyTorchLinearSVM using CF_SingleMatch with counterfactual by GlobeCE
INFO:root:Computing max_mean_discrepancy for (PyTorchLinearSVM, GlobeCE, CF_SingleMatch)
100%|██████████| 100/100 [00:05<00:00, 17.66it/s]
INFO:root:Policy for PyTorchLinearSVM using CF_OTMatch_2.0 with counterfactual by GlobeCE
INFO:root:Computing max_mean_discrepancy for (PyTorchLinearSVM, GlobeCE, CF_OTMatch_2.0)
100%|██████████| 100/100 [00:07<00:00, 13.15it/s]


In [ ]:
from experiments import plotting

plotting.intervention_vs_distance(experiment, save_to_file=False)

In [146]:
data = experiment.distance_results['PyTorchLinearSVM']['GlobeCE']

In [147]:
data['CF_UniformMatch'].keys()

dict_keys(['max_mean_discrepancy'])

In [148]:
from experiments.latex import TikzPlotGenerator

In [149]:
plot_generator = TikzPlotGenerator(data)
print(plot_generator.generate_plot_code(
    metric='max_mean_discrepancy', 
    methods=[
        "Train_Distri",
        'CF_UniformMatch',
        'CF_SingleMatch',
        'CF_OTMatch_2.0',
    ]
))


\begin{tikzpicture}
\begin{axis}[
    width=4.4cm, height=4.5cm,
    legend pos=south west,
    legend style={
        draw=none,
    font=\scriptsize,
        legend image code/.code={
            \draw[mark repeat=2,mark phase=2]
                plot coordinates {
                    (0cm,0cm)
                    (0.2cm,0cm) % Adjust the length here
                };
        },
    },
    legend cell align={left},
    grid=major,
    title={SVM, $\A_{\text{CE}}=$GlobeCE},
    title style={font=\footnotesize, yshift=-1ex}, 
    font=\footnotesize,
    % Increase margin of x and y tick labels
    xticklabel style={xshift=-0pt, yshift=-3pt},
    yticklabel style={xshift=-3pt, yshift=0pt},
]
        \addplot[name path=TR-Uni-MMD-line, blue, thick] coordinates {
    (0, 0.48448240756988525)
    (20, 0.4658566093444824)
    (40, 0.4510882031917572)
    (60, 0.4360354149341583)
    (80, 0.4195618438720703)
    (100, 0.4013877058029175)
    (120, 0.38892304062843325)
    (140, 0.3794191765